### 1. Import relevant libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns


from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from statsmodels.graphics.api import abline_plot

import warnings
warnings.filterwarnings(action='ignore')

### 2. Load the data

In [ ]:
# Define file paths for loading the data
train_data_path = 'train_data.csv'
test_data_path = 'test_data.csv'
train_target_path = 'train_target.csv'
test_target_path = 'test_target.csv'

# Load training and testing features from CSV files
X_train = pd.read_csv(train_data_path)
X_test = pd.read_csv(test_data_path)

# Load training and testing target variables from CSV files
y_train = pd.read_csv(train_target_path)['WQI']
y_test = pd.read_csv(test_target_path)['WQI'] 

In [ ]:
# Display the summary information of the loaded data
X_train.info()

In [ ]:
X_test.info()

### 3. Modeling

Based on the information provided by info(), it seems that both the training and testing datasets have been loaded successfully without any missing values. Each dataset contains 10 columns, all of which are numerical (float64), including the target variable 'WQI'. There are 820 samples in the training set and 205 samples in the testing set, with 10 features each. 

Since the target variable, WQI, is continuous, Linear Regression, Random Forest Regression, and XGBoost Regression model are used to explore different approaches and leverage the strengths of each method.

#### 3a. Making a Linear Regression Model: first model

In [ ]:
# Define the Linear Regression model
linear_model = LinearRegression()

# Fit the model to the training data
linear_model.fit(X_train, y_train)

# Predict using the model
y_pred_linear = linear_model.predict(X_test)

# Evaluate the model
r2_linear = r2_score(y_test, y_pred_linear)
mse_score_linear = mean_squared_error(y_test, y_pred_linear)
mae_linear = mean_absolute_error(y_test, y_pred_linear)

print("Linear Regression:")
print("R-squared (R2) on Testing Set:", r2_linear)
print("Mean Squared Error (MSE) on Testing Set:", mse_score_linear)
print("Mean Absolute Error (MAE) on Testing Set:", mae_linear)


In [ ]:
# Define the range of number of splits
num_splits_range = [3, 5, 7, 10]

# Dictionary to store cross-validation scores for each number of splits
cv_scores_linear_test = {}
cv_scores_linear_train = {}

for num_splits in num_splits_range:
    # Define cross-validation strategy with current number of splits
    kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

    # Perform cross-validation for Linear Regression
    cv_scores_linear_test[num_splits] = cross_val_score(linear_model, X_test, y_test, cv=kf, scoring='r2')
    cv_scores_linear_train[num_splits] = cross_val_score(linear_model, X_train, y_train, cv=kf, scoring='r2')
  
    # Print the cross-validation scores for each number of splits
    print(f"{num_splits}-fold Cross-Validation Scores for Linear Regression:")
    print("Mean cross-validation score on testing set:", np.mean(cv_scores_linear_test[num_splits]))
    print("Mean cross-validation score on training set:", np.mean(cv_scores_linear_train[num_splits]))
    print("Standard deviation in cross-validation scores on testing set:", np.std(cv_scores_linear_test[num_splits]))
    print()

In [ ]:
# Plot the predictions against the actual values for Linear Regression
plt.scatter(y_test, y_pred_linear)
plt.plot(y_test, y_test, color='red', linestyle='--')  # Plot the perfect fit line
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Predictions vs Actual Values (Linear Regression)')
plt.show()

In [ ]:
# Define the Lasso model
lasso_model = Lasso(alpha=0.1)

# Fit the Lasso model to the training data
lasso_model.fit(X_train, y_train)

# Predict using the Lasso model
y_pred_lasso = lasso_model.predict(X_test)

# Evaluate the Lasso model
lasso_r2 = r2_score(y_test, y_pred_lasso)
lasso_mse = mean_squared_error(y_test, y_pred_lasso)
lasso_mae = mean_absolute_error(y_test, y_pred_lasso)

print("Lasso Regression:")
print("R-squared (R2) on Testing Set:", lasso_r2)
print("Mean Squared Error (MSE) on Testing Set:", lasso_mse)
print("Mean Absolute Error (MAE) on Testing Set:", lasso_mae)

In [ ]:
# Define the Ridge model
ridge_model = Ridge(alpha=0.1)

# Fit the Ridge model to the training data
ridge_model.fit(X_train, y_train)

# Predict using the Ridge model
y_pred_ridge = ridge_model.predict(X_test)

# Evaluate the Ridge model
ridge_r2 = r2_score(y_test, y_pred_ridge)
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_mae = mean_absolute_error(y_test, y_pred_ridge)

print("Ridge Regression:")
print("R-squared (R2) on Testing Set:", ridge_r2)
print("Mean Squared Error (MSE) on Testing Set:", ridge_mse)
print("Mean Absolute Error (MAE) on Testing Set:", ridge_mae)

#### 3b. Making a Random Forest Regression Methods: second model

In [ ]:
# Define the Random Forest Regression model
rf_model = RandomForestRegressor()

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Predict using the model
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_score_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print("Random Forest Regression:")
print("R-squared (R2) on Testing Set:", r2_rf)
print("Mean Squared Error (MSE) on Testing Set:", mse_score_rf)
print("Mean Absolute Error (MAE) on Testing Set:", mae_rf)

In [ ]:
# Dictionary to store cross-validation scores for each number of splits
cv_scores_rf_test = {}
cv_scores_rf_train = {}

# Define the Random Forest Regression model
rf_model = RandomForestRegressor()

for num_splits in num_splits_range:
    # Define cross-validation strategy with current number of splits
    kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    # Perform hyperparameter tuning with GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2']
    }
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=kf, scoring='r2')
    grid_search.fit(X_train, y_train)
    
    # Get the best model
    best_rf_model = grid_search.best_estimator_
    
    # Evaluate the best model using cross-validation on the training set
    cv_scores_rf_train[num_splits] = cross_val_score(best_rf_model, X_train, y_train, cv=kf, scoring='r2')
    
    # Evaluate the best model using cross-validation on the testing set
    cv_scores_rf_test[num_splits] = cross_val_score(best_rf_model, X_test, y_test, cv=kf, scoring='r2')

# Print the cross-validation scores for each number of splits
for num_splits in num_splits_range:
    print(f"{num_splits}-fold Cross-Validation Scores:")
    print("Mean cross-validation score on training set:", np.mean(cv_scores_rf_train[num_splits]))
    print("Mean cross-validation score on testing set:", np.mean(cv_scores_rf_test[num_splits]))
    print("Standard deviation in cross-validation scores on testing set:", np.std(cv_scores_rf_test[num_splits]))

In [ ]:
# Plot the predictions against the actual values
plt.scatter(y_test, y_pred_rf, color='blue', label='Predictions')
plt.plot(y_test, y_test, color='red', linestyle='--', label='Perfect Fit')
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Predictions vs Actual Values for Random Forest Regression')
plt.legend()
plt.show()

#### 3d. Making a XGBoost Regression Model: third model

In [ ]:
# Define the Random Forest Regression model
xgb_model = xgb.XGBRegressor()

# Fit the model to the training data
xgb_model.fit(X_train, y_train)

# Predict using the model
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model
r2_xgb = r2_score(y_test, y_pred_xgb)
mse_score_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)

print("XGBoost Regression:")
print("R-squared (R2) on Testing Set:", r2_xgb)
print("Mean Squared Error (MSE) on Testing Set:", mse_score_xgb)
print("Mean Absolute Error (MAE) on Testing Set:", mae_xgb)

In [ ]:
# Dictionary to store cross-validation scores for each number of splits
cv_scores_xgb_test = {}
cv_scores_xgb_train = {}

# Define the XGBoost Regression model
xgb_model = xgb.XGBRegressor()

for num_splits in num_splits_range:
    # Define cross-validation strategy with current number of splits
    kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    # Perform hyperparameter tuning with GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.05, 0.1, 0.15],
        'min_child_weight': [1, 2, 3],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9]
    }
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=kf, scoring='r2')
    grid_search.fit(X_train, y_train)
    
    # Get the best model
    best_xgb_model = grid_search.best_estimator_
    
    # Evaluate the best model using cross-validation on the training set
    cv_scores_xgb_train[num_splits] = cross_val_score(best_xgb_model, X_train, y_train, cv=kf, scoring='r2')
    
    # Evaluate the best model using cross-validation on the testing set
    cv_scores_xgb_test[num_splits] = cross_val_score(best_xgb_model, X_test, y_test, cv=kf, scoring='r2')

# Print the cross-validation scores for each number of splits
for num_splits in num_splits_range:
    print(f"{num_splits}-fold Cross-Validation Scores:")
    print("Mean cross-validation score on training set:", np.mean(cv_scores_xgb_train[num_splits]))
    print("Mean cross-validation score on testing set:", np.mean(cv_scores_xgb_test[num_splits]))
    print("Standard deviation in cross-validation scores on testing set:", np.std(cv_scores_xgb_test[num_splits]))

In [ ]:
# Plot the predictions against the actual values
plt.scatter(y_test, y_pred_xgb, color='blue', label='Predictions')
plt.plot(y_test, y_test, color='red', linestyle='--', label='Perfect Fit')
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Predictions vs Actual Values for XGBoost Regression')
plt.legend()
plt.show()

### 4. Model Evaluation

### 5. Final Model